In [ ]:
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.retrievers import WikipediaRetriever
load_dotenv("../.env")
import os
from langchain.vectorstores import Chroma
import numpy as np  

In [10]:
wiki_retriever = WikipediaRetriever()
docs = wiki_retriever.invoke("Python")

/home/kamesh/github-projects/langchain-tutorials-campusx/.venv/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/kamesh/github-projects/langchain-tutorials-campusx/.venv/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


# Using vector store

In [11]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]


In [14]:
# Initialize Ollama embeddings
embeddings_model = OllamaEmbeddings(
    model=os.getenv("OLLAMA_EMBEDDINGS_MODEL"),
)

In [ ]:
# shape of the embedding
np.array(embeddings_model.embed_query(documents[0].page_content)).shape

(768,)

In [ ]:
# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory="../store/chroma_db",
    collection_name="my_collection"
)

In [ ]:
# getting all th collection
vectorstore.get()

{'ids': ['4d10d6c1-ca58-46ce-91ff-6f15e272c398',
  '53e564dd-c34e-4f92-a76f-57c6e6802f7e',
  'e1997b7b-e1b9-4034-aac3-c6a715a18750',
  'ad3ef189-bef7-408c-a2fa-16aa64a09c08'],
 'embeddings': None,
 'documents': ['LangChain helps developers build LLM applications easily.',
  'Chroma is a vector database optimized for LLM-based search.',
  'Embeddings convert text into high-dimensional vectors.',
  'OpenAI provides powerful embedding models.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None, None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [34]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3,"lambda_mult":0.2},
                                     seartch_type="mmr",)

In [32]:
vectorstore.similarity_search_with_score(
    query="What is LangChain?",
    k=2,
)

[(Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.'),
  0.4292033706976669),
 (Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
  1.0267069683738397)]

In [35]:
retriever.invoke("What chroma?")

TypeError: Collection.query() got an unexpected keyword argument 'lambda_mult'

In [22]:
# Connect to the Chroma vector store
chroma_db = Chroma(
    persist_directory="../store/chroma_db",  # Replace with the actual path to your Chroma DB
    collection_name="my_collection",
    embedding_function=embeddings_model
)

In [ ]:
# List all collections in the Chroma vector store
collections = chroma_db.get()
collections

{'ids': ['582b249d-b397-41b2-81e9-4e17bec00b34',
  'd8863c91-ee25-469e-820c-a2ac95133339',
  '14f10c62-4b69-4429-aef5-e55e4912de54',
  '326121de-038c-4a00-8a38-d2ff67442478'],
 'embeddings': None,
 'documents': ['LangChain helps developers build LLM applications easily.',
  'Chroma is a vector database optimized for LLM-based search.',
  'Embeddings convert text into high-dimensional vectors.',
  'OpenAI provides powerful embedding models.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None, None]}